### Downloading dependensies

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install pycaret
from pycaret.time_series import *
!pip install scikit-learn

In [4]:
import pickle
from pycaret.classification import *

### Accessing Dataset

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sms_spam.csv')
df

,type,text
0,nonspam,"Go until jurong point, crazy.. Available only ..."
1,nonspam,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,nonspam,U dun say so early hor... U c already then say...
4,nonspam,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,nonspam,Will ü b going to esplanade fr home?
5571,nonspam,"Pity, * was in mood for that. So...any other s..."
5572,nonspam,The guy did some bitching but I acted like i'd...


### Data Preprocessing-

In [16]:
#SETTING UP THE DATA FOR MODELLING
setup(data=df, target='type')

,Description,Value
0,Session id,8063
1,Target,type
2,Target type,Binary
3,Target mapping,"nonspam: 0, spam: 1"
4,Original data shape,"(5574, 2)"
5,Transformed data shape,"(5574, 2)"
6,Transformed train set shape,"(3901, 2)"
7,Transformed test set shape,"(1673, 2)"
8,Categorical features,1
9,Preprocess,True


### Converting Strings as Numbers

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sms_spam.csv')

# Drop rows with missing values in 'text' column
df['text'].fillna('missing', inplace=True)
df.dropna(subset=['text'], inplace=True)

# Split dataset into features (X) and target variable (y)
X = df.drop(columns=['type'])
y = df['type']

# Split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

X_train_text = tfidf_vectorizer.fit_transform(X_train['text'])
X_test_text = tfidf_vectorizer.transform(X_test['text'])

X_train_text_df = pd.DataFrame(X_train_text.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
X_test_text_df = pd.DataFrame(X_test_text.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

X_train_processed = pd.concat([X_train.drop(columns=['text']), X_train_text_df], axis=1)
X_test_processed = pd.concat([X_test.drop(columns=['text']), X_test_text_df], axis=1)

print("X_train_processed:")
print(X_train_processed.head())

print("\nX_test_processed:")
print(X_test_processed.head())


X_train_processed:
       00  000  000pes  008704050406  0089  0121  01223585236  01223585334  \
1350  0.0  0.0     0.0           0.0   0.0   0.0          0.0          0.0   
5544  NaN  NaN     NaN           NaN   NaN   NaN          NaN          NaN   
1168  0.0  0.0     0.0           0.0   0.0   0.0          0.0          0.0   
5551  NaN  NaN     NaN           NaN   NaN   NaN          NaN          NaN   
5320  NaN  NaN     NaN           NaN   NaN   NaN          NaN          NaN   

       02  0207  ...  zhong  zindgi  zoe  zogtorius  zoom  zouk  zyada   èn  \
1350  0.0   0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0  0.0   
5544  NaN   NaN  ...    NaN     NaN  NaN        NaN   NaN   NaN    NaN  NaN   
1168  0.0   0.0  ...    0.0     0.0  0.0        0.0   0.0   0.0    0.0  0.0   
5551  NaN   NaN  ...    NaN     NaN  NaN        NaN   NaN   NaN    NaN  NaN   
5320  NaN   NaN  ...    NaN     NaN  NaN        NaN   NaN   NaN    NaN  NaN   

       ú1  〨ud  
1350  0.0  0.0  
554

In [69]:
print(y_train.value_counts())


type
nonspam    3873
spam        586
Name: count, dtype: int64


## Using pycaret to select the best Model-

In [70]:
from pycaret.classification import *

exp_clf = setup(data=df, target='type', train_size=0.8, session_id=110)

# Compare different classification models
best_model = compare_models()
evaluate_model(best_model)



,Description,Value
0,Session id,110
1,Target,type
2,Target type,Binary
3,Target mapping,"nonspam: 0, spam: 1"
4,Original data shape,"(5574, 2)"
5,Transformed data shape,"(5574, 2)"
6,Transformed train set shape,"(4459, 2)"
7,Transformed test set shape,"(1115, 2)"
8,Categorical features,1
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8953,0.6426,0.8953,0.9066,0.8642,0.3244,0.4387,0.8950
knn,K Neighbors Classifier,0.8953,0.6096,0.8953,0.9066,0.8642,0.3244,0.4387,0.2050
nb,Naive Bayes,0.8953,0.6426,0.8953,0.9066,0.8642,0.3244,0.4387,0.1170
dt,Decision Tree Classifier,0.8953,0.6096,0.8953,0.9066,0.8642,0.3244,0.4387,0.1240
svm,SVM - Linear Kernel,0.8953,0.6426,0.8953,0.9066,0.8642,0.3244,0.4387,0.0690
ridge,Ridge Classifier,0.8953,0.6426,0.8953,0.9066,0.8642,0.3244,0.4387,0.0670
rf,Random Forest Classifier,0.8953,0.6096,0.8953,0.9066,0.8642,0.3244,0.4387,0.2440
qda,Quadratic Discriminant Analysis,0.8953,0.6426,0.8953,0.9066,0.8642,0.3244,0.4387,0.0660
ada,Ada Boost Classifier,0.8953,0.6096,0.8953,0.9066,0.8642,0.3244,0.4387,0.0850
gbc,Gradient Boosting Classifier,0.8953,0.6096,0.8953,0.9066,0.8642,0.3244,0.4387,0.1880


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Best model appears to be the Logistic Regression classifier.

In [71]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sms_spam.csv')

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Split dataset into features (X) and target variable (y)
X = df['text']
y = df['type']
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

#logistic regression model
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train_tfidf, y_train)

accuracy = logistic_regression_model.score(X_test_tfidf, y_test)
print("Logistic Regression Model Accuracy:", accuracy)


Logistic Regression Model Accuracy: 0.9721973094170404


### Testing the model with some examples-

In [72]:
example_texts = [
    "Congratulations! You've won a free vacation. Click here to claim your prize.",
    "Hello, this is just a regular message.",
    "URGENT: Your account has been suspended. Please click the link to verify your identity.",
    "Hey, what's up?",
    "Limited time offer: Get 50% off on all purchases. Shop now!"
]

example_texts_tfidf = tfidf_vectorizer.transform(example_texts)
predictions = logistic_regression_model.predict(example_texts_tfidf)

for text, prediction in zip(example_texts, predictions):
    print(f"Text: {text}")
    print(f"Predicted Label: {'spam' if prediction == 'spam' else 'non-spam'}")
    print()


Text: Congratulations! You've won a free vacation. Click here to claim your prize.
Predicted Label: spam

Text: Hello, this is just a regular message.
Predicted Label: non-spam

Text: URGENT: Your account has been suspended. Please click the link to verify your identity.
Predicted Label: spam

Text: Hey, what's up?
Predicted Label: non-spam

Text: Limited time offer: Get 50% off on all purchases. Shop now!
Predicted Label: non-spam

